# Projetando um app de controle de finanças

A ideia é construir uma aplicação que ajude a ver para onde o dinheiro está indo.

* Definição de metas;
* Cadastro de contas;
* Cadastro de cartões;
* Enviar e-mail;


### Models

In [1]:
import enum
import os

import datetime

from sqlalchemy import (Column, DateTime, Enum, ForeignKey, Integer, Numeric,
                        String, Table, create_engine, select)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, scoped_session, sessionmaker



In [2]:
Base = declarative_base()


class TransactionType(enum.Enum):
    """Enum."""

    INCOME = 0
    OUTGO = 1
    TRANSFER = 2


class PaymentType(enum.Enum):
    
    CASH = 0
    DEBIT = 1
    CREDIT = 2


class MySession():
    """Sessão com BD.
    Para definir a sessão com o BD na aplicação. Para os
    testes, passando o parâmetro test=True, um BD na memória
    """

    def __init__(self, base, test=False):
        """Inicializa."""
        if test:
            path = ':memory:'
        else:
            path = ':memory:'
        self._engine = create_engine('sqlite:///' + path, convert_unicode=True)
        Session = sessionmaker(bind=self._engine)
        if test:
            self._session = Session()
        else:
            self._session = scoped_session(Session)
            base.metadata.bind = self._engine

    @property
    def session(self):
        """Session."""
        return self._session

    @property
    def engine(self):
        """Engine."""
        return self._engine
    
    
class User(Base):
    __tablename__ = 'user'
    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True)
    _password = Column(String(30))
    
    def __init__(self, username, password):
        self.username = username
        self._password = password
        
    def __repr__(self):
        return f'User {self.username}'


account_transaction = Table('Account_Transaction', Base.metadata,
                           Column('account_id', Integer, ForeignKey('account.id')),
                           Column('transaction_id', Integer, ForeignKey('transaction.id')))


class Account(Base):
    """docstring for Account"""
    __tablename__ = 'account'
    id = Column(Integer, primary_key=True)
    bank = Column(String(50))
    description = Column(String(50), unique=True)
    balance = Column(Numeric(asdecimal=False))
    user_id = Column(Integer, ForeignKey('user.id'))
    creditcard_id = Column(Integer, ForeignKey(
        "creditcard.id"))
    transaction = relationship('Transaction',
                              secondary=account_transaction,
                              backref='account')

    def __init__(self, description, bank, balance, user, creditcard = None):
        self.description = description
        self.bank = bank
        self.balance = balance
        self.user_id = user.id
        if(creditcard):
            self.creditcard_id = creditcard.id


class Transaction(Base):
    """docstring for Transaction"""
    __tablename__ = 'transaction'
    id = Column(Integer, primary_key=True)
    description = Column(String(50))
    value = Column(Numeric(asdecimal=False))
    date = Column(DateTime)
    payed_with = Column(Enum(PaymentType))
    parcels = Column(Integer, default=1)
    category_id = Column(Integer, ForeignKey('category.id'))
    transaction_type = Column(Enum(TransactionType))
    
    def __init__(self, description, value, date, payed_with, parcels, category, transaction_type):
        self.description = description
        self.value = value
        self.date = date
        self.payed_with = payed_with
        self.parcels = parcels
        self.category_id = category.id
        self.transaction_type = transaction_type


class Category(Base):
    """docstring for Categoria"""
    __tablename__ = 'category'
    id = Column(Integer, primary_key=True)
    description = Column(String(50), unique=True)

    def __init__(self, description):
        self.description = description


class CreditCard(Base):
    __tablename__ = 'creditcard'
    id = Column(Integer, primary_key=True)
    limit = Column(Numeric(asdecimal=False))
    date_pay = Column(DateTime)
    
    def __init__(self, limit, account, date_pay):
        self.limit = limit
        self.account_id = account.id
        self.date_pay = date_pay
        




In [3]:
#----------------------------
# Turn Foreign Key Constraints ON for
# each connection.
#----------------------------

from sqlalchemy.engine import Engine
from sqlalchemy import event

@event.listens_for(Engine, "connect")
def set_sqlite_pragma(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")
    cursor.close()

#----------------------------
# Create the engine
#----------------------------

#----------------------------
# Create the Schema
#----------------------------

mysession = MySession(Base)
dbsession = mysession.session
engine = mysession.engine

Base.metadata.create_all(engine)

#----------------------------
# Create the Session class 
#----------------------------



# from sqlalchemy.orm import sessionmaker
# Session = sessionmaker(bind=engine)

#----------------------------
# Populate the database 
#----------------------------


In [4]:
def user_registration(username:str, password:str):
    user = User(username, password)
    dbsession.add(user)
    dbsession.commit()

def create_account(bank:str, description:str, balance:str, user:User):
    account = Account(description, bank, balance, user)
    dbsession.add(account)
    dbsession.commit()

def add_creditcard(limit:float, account:Account, date_pay:datetime):
    creditcard = CreditCard(limit, account, date_pay)
    dbsession.add(creditcard)
    dbsession.commit()

def create_category(description:str):
    category = Category(description)
    dbsession.add(category)
    dbsession.commit()

def add_transaction(description, value, date, payed_with, parcels, category, transaction_type):
    transaction = Transaction(description, value, date, payed_with, parcels, category, transaction_type)
    dbsession.add(transaction)
    dbsession.commit()


In [5]:
user_registration('gi', 'senha')

usern = dbsession.query(User).filter(User.username == 'gi').first()

create_account('Itaú', 'Conta', 500, usern)

accountn = dbsession.query(Account).all()

In [6]:
create_category('Moradia')

In [7]:
cat = dbsession.query(Category).first()
cat

In [8]:
datetime.datetime.now()

datetime.datetime(2019, 5, 18, 16, 5, 19, 256563)

In [9]:
add_transaction('teste', 20, datetime.datetime.now(), PaymentType.CASH, 1, cat, TransactionType.OUTGO)

In [10]:
acc = dbsession.query(Account).join(User).filter(User.id == Account.user_id).first()
add_creditcard(1000, acc, datetime.datetime.now())

In [11]:
vars(dbsession.query(Account).join(User).filter(User.id == Account.user_id).first())

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f8da6e44898>,
 'description': 'Conta',
 'id': 1,
 'creditcard_id': None,
 'balance': 500,
 'bank': 'Itaú',
 'user_id': 1}

In [12]:
vars(dbsession.query(Account).filter(Account.user_id == 1).first())

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f8da6e44898>,
 'description': 'Conta',
 'id': 1,
 'creditcard_id': None,
 'balance': 500,
 'bank': 'Itaú',
 'user_id': 1}

In [13]:
trans = dbsession.query(Transaction).first()

In [14]:
vars(trans)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f8da6e6f710>,
 'category_id': 1,
 'payed_with': <PaymentType.CASH: 0>,
 'date': datetime.datetime(2019, 5, 18, 16, 5, 19, 340432),
 'description': 'teste',
 'transaction_type': <TransactionType.OUTGO: 1>,
 'parcels': 1,
 'value': 20,
 'id': 1}

In [15]:
acc.transaction.append(trans)

In [16]:
dbsession.query(account_transaction).all()

[(1, 1)]

In [17]:
for t in acc.transaction:
    print(vars(t))

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f8da6e6f710>, 'category_id': 1, 'payed_with': <PaymentType.CASH: 0>, 'date': datetime.datetime(2019, 5, 18, 16, 5, 19, 340432), 'description': 'teste', 'transaction_type': <TransactionType.OUTGO: 1>, 'parcels': 1, 'value': 20, 'id': 1}


In [ ]:
from flask_bcrypt import Bcrypt